In [ ]:
# 顧客流失偵測

In [ ]:
## 載入套件與讀取資料

In [ ]:
### 載入套件

In [1]:
# 載入所需套件

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
'''
圖形中有中文字型的問題
參考
https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/359974/
'''
from matplotlib.font_manager import FontProperties

han_font = FontProperties(fname=r"c:/windows/fonts/msjh.ttc", size=14) # 中文字形

In [3]:
'''
設計圖形呈現的外觀風格
'''
sns.set(style="whitegrid")

In [ ]:
### 讀取資料

In [4]:
# 讀入資料檔
df = pd.read_excel('Online Retail.xlsx')

In [ ]:
### 資料清理

In [5]:
# 去除CustomerID沒有資料的紀錄
df = df.dropna(subset=['CustomerID'])

In [6]:
from datetime import datetime

# 取出2010-12-09到2011-12-09一年之間的資料
df = df[df.InvoiceDate>=datetime(2010, 12, 9, 0, 0, 0)]

In [7]:
#取出購買紀錄(不包含取消紀錄)
df = df[df.Quantity>0] 

In [ ]:
### 一年中的顧客購買次數

In [8]:
# 將同一個顧客的發票紀錄聚集成群後，計算不重複的發票編號數量，按照數量由大到小排序
CustomerData = df.groupby("CustomerID").agg({"InvoiceNo": "nunique"})\
.reset_index().sort_values("InvoiceNo", ascending=False)

CustomerData.head(10)

,CustomerID,InvoiceNo
322,12748.0,196
1859,14911.0,194
3960,17841.0,121
554,13089.0,94
1644,14606.0,90
2152,15311.0,88
475,12971.0,83
1672,14646.0,74
785,13408.0,60
2668,16029.0,60


In [9]:
# 一年中購買次數20次或以上的常客
MajorCustomers = CustomerData.CustomerID[CustomerData.InvoiceNo>=20].values

In [10]:
# 常客的購物紀錄
mc_df = df.loc[df.CustomerID.isin(MajorCustomers), ]

In [20]:
# 如果同一筆交易分在連續兩個時間內完成，則取最大者
mc_df = mc_df.groupby(["CustomerID", "InvoiceNo"])\
.agg({"InvoiceDate": "max"}).reset_index()

In [25]:
# 將時間計算從timestamp格式換成datetime格式
mc_freq = mc_df.assign(InvoiceTime=mc_df.InvoiceDate.apply(lambda x: pd.to_datetime(x)))

In [27]:
mc_freq = mc_freq.groupby(["CustomerID", "InvoiceTime"])\
.InvoiceNo.nunique().reset_index()

In [28]:
mc_freq.head(50)

,CustomerID,InvoiceTime,InvoiceNo
0,12415.0,2011-01-06 11:12:00,1
1,12415.0,2011-01-10 09:58:00,1
2,12415.0,2011-02-15 09:52:00,1
3,12415.0,2011-03-03 10:59:00,1
4,12415.0,2011-04-01 14:28:00,1
5,12415.0,2011-05-17 15:42:00,1
6,12415.0,2011-05-20 14:13:00,1
7,12415.0,2011-06-15 13:37:00,2
8,12415.0,2011-07-13 15:30:00,1
9,12415.0,2011-07-13 15:31:00,1


In [29]:
mc_freq = mc_freq\
.assign(TimeDiff = mc_freq.groupby("CustomerID").InvoiceTime.diff())

In [30]:
mc_freq.head(50)

,CustomerID,InvoiceTime,InvoiceNo,TimeDiff
0,12415.0,2011-01-06 11:12:00,1,NaT
1,12415.0,2011-01-10 09:58:00,1,3 days 22:46:00
2,12415.0,2011-02-15 09:52:00,1,35 days 23:54:00
3,12415.0,2011-03-03 10:59:00,1,16 days 01:07:00
4,12415.0,2011-04-01 14:28:00,1,29 days 03:29:00
5,12415.0,2011-05-17 15:42:00,1,46 days 01:14:00
6,12415.0,2011-05-20 14:13:00,1,2 days 22:31:00
7,12415.0,2011-06-15 13:37:00,2,25 days 23:24:00
8,12415.0,2011-07-13 15:30:00,1,28 days 01:53:00
9,12415.0,2011-07-13 15:31:00,1,0 days 00:01:00
